## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import json
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Uchaly,RU,54.2996,59.4521,15.06,95,100,4.14,overcast clouds
1,1,Albany,US,42.6001,-73.9662,26.91,87,100,6.33,light snow
2,2,Port Alfred,ZA,-33.5906,26.8910,68.16,80,100,7.52,light rain
3,3,Ushuaia,AR,-54.8000,-68.3000,62.91,41,33,2.42,scattered clouds
4,4,Cabo San Lucas,MX,22.8909,-109.9124,74.52,48,0,8.05,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] <=max_temp)&
                                    (city_data_df["Max Temp"]>= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Hithadhoo,MV,-0.6000,73.0833,81.99,73,20,10.09,few clouds
25,25,San Andres,CO,12.5847,-81.7006,80.71,70,12,13.24,few clouds
26,26,Hilo,US,19.7297,-155.0900,77.61,71,100,5.75,overcast clouds
30,30,Avarua,CK,-21.2078,-159.7750,80.65,78,40,11.50,scattered clouds
41,41,Kampot,KH,10.6167,104.1833,77.95,76,70,3.15,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
6,6,Hithadhoo,MV,-0.6000,73.0833,81.99,73,20,10.09,few clouds
25,25,San Andres,CO,12.5847,-81.7006,80.71,70,12,13.24,few clouds
26,26,Hilo,US,19.7297,-155.0900,77.61,71,100,5.75,overcast clouds
30,30,Avarua,CK,-21.2078,-159.7750,80.65,78,40,11.50,scattered clouds
41,41,Kampot,KH,10.6167,104.1833,77.95,76,70,3.15,broken clouds
...,...,...,...,...,...,...,...,...,...,...
696,696,Ranong,TH,9.9658,98.6348,75.94,89,66,5.99,broken clouds
698,698,Sorong,ID,-0.8833,131.2500,80.69,80,99,10.80,overcast clouds
700,700,Cayenne,GF,4.9333,-52.3333,82.44,82,75,5.99,broken clouds
701,701,Bandarbeyla,SO,9.4942,50.8122,75.38,64,0,10.71,clear sky


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Hithadhoo,MV,81.99,few clouds,-0.6000,73.0833,
25,San Andres,CO,80.71,few clouds,12.5847,-81.7006,
26,Hilo,US,77.61,overcast clouds,19.7297,-155.0900,
30,Avarua,CK,80.65,scattered clouds,-21.2078,-159.7750,
41,Kampot,KH,77.95,broken clouds,10.6167,104.1833,
42,Carnarvon,AU,80.67,broken clouds,-24.8667,113.6333,
47,Limbang,MY,80.71,overcast clouds,4.7500,115.0000,
58,Puerto Ayora,EC,82.36,broken clouds,-0.7393,-90.3518,
59,Cabedelo,BR,83.32,scattered clouds,-6.9811,-34.8339,
60,Lat Yao,TH,80.51,overcast clouds,15.7510,99.7893,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"]=f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df['Hotel Name'].replace('', np.nan, inplace=True)   
        

In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   199
Country                199
Max Temp               199
Current Description    199
Lat                    199
Lng                    199
Hotel Name             185
dtype: int64

In [11]:
clean_hotel_df=hotel_df.dropna()
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig=gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))